In [3]:
from FlagEmbedding import BGEM3FlagModel
from tqdm.notebook import tqdm
from dotenv import load_dotenv

load_dotenv()

model = BGEM3FlagModel("BAAI/bge-m3", use_fp16=True)

/Users/I551385/Library/Caches/pypoetry/virtualenvs/ie691-ir-project-u22LL72s-py3.12/lib/python3.12/site-packages/accelerate/utils/other.py:220: DeprecationWarning: numpy.core is deprecated and has been renamed to numpy._core. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.multiarray.
  np.core.multiarray._reconstruct,


Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

In [74]:
import weaviate
import weaviate.classes as wvc
#client = weaviate.connect_to_custom(http_host=os.getenv("WEAVIATE_HTTP_HOST"),http_port=int(os.getenv("WEAVIATE_HTTP_PORT")), http_secure=True, grpc_host=os.getenv("WEAVIATE_GRPC_HOST"), grpc_port=int(os.getenv("WEAVIATE_GRPC_PORT")), grpc_secure=True, auth_credentials=weaviate.auth.AuthApiKey(api_key=os.getenv("WEAVIATE_API_KEY")))
client = weaviate.connect_to_local()

In [76]:
#client.collections.delete("neuclir_1_mutli_bge_m3")

In [77]:
try:
    collection = client.collections.get("neuclir_1_mutli_bge_m3")
    documents = client.collections.create(
        name="neuclir_1_mutli_bge_m3",
        vectorizer_config=[
            wvc.config.Configure.NamedVectors.none(
                name="title_dense",
                vector_index_config=wvc.config.Configure.VectorIndex.hnsw(
                    vector_cache_max_objects=100000,
                    #quantizer=wvc.config.Reconfigure.VectorIndex.Quantizer.pq(training_limit=100000)  # Set the threshold to begin training
                ),
                
            ),
            wvc.config.Configure.NamedVectors.none( 
                name="text_dense",
                vector_index_config=wvc.config.Configure.VectorIndex.hnsw(
                    vector_cache_max_objects=100000,
                   # quantizer=wvc.config.Reconfigure.VectorIndex.Quantizer.pq(training_limit=10000)  # Set the threshold to begin training
                ),
            )],
        properties=[
            wvc.config.Property(
                name="doc_id",
                data_type=wvc.config.DataType.UUID,
            ),
            wvc.config.Property(
                name="title_sparse",
                data_type=wvc.config.DataType.BLOB,
            ),
              wvc.config.Property(
                name="document_sparse",
                data_type=wvc.config.DataType.BLOB,
            ),
            wvc.config.Property(
                name="title_colbert",
                data_type=wvc.config.DataType.BLOB,
            ),
           wvc.config.Property(
                name="document_colbert",
                data_type=wvc.config.DataType.BLOB,
            ),
            wvc.config.Property(
                name="title",
                data_type=wvc.config.DataType.TEXT,
            ),
            wvc.config.Property(
                name="text",
                data_type=wvc.config.DataType.TEXT,
            ),
            wvc.config.Property(
                name="url",
                data_type=wvc.config.DataType.TEXT,
            )
        ])
except Exception as e:
    print(e)
    

In [27]:
import pandas as pd
from explore.funcs import load_datasets

datasets = load_datasets(["zh", "ru", "fa"])

docs = pd.concat([pd.DataFrame(dataset.docs_iter()) for dataset in datasets.values()])


len(docs)

1876367

In [10]:
import base64
def to_blob(obj):
    return base64.b64encode(pickle.dumps(obj)).decode('utf-8')

In [45]:
import pickle

batches = [docs[i:i+10000] for i in range(100000, len(docs), 10000)]
zh = client.collections.get("hc4_filtered_bge_m3")

outer_progress = tqdm(total=len(docs))

for i, batch in enumerate(batches):
    title_embeddings = model.encode(batch["title"].to_list(), return_dense=True, return_sparse=False, return_colbert_vecs=False)
    doc_embeddings = model.encode(batch["text"].to_list(), return_dense=True, return_sparse=False, return_colbert_vecs=False)
    #title_sparse_blobs = [to_blob(x) for x in title_embeddings["lexical_weights"]]
    #title_colbert_blobs = [to_blob(x) for x in title_embeddings["colbert_vecs"]]
    batch = batch.reset_index(drop=True)
    with zh.batch.fixed_size(100, 2) as b:
        for row in batch.itertuples(index=True):
            b.add_object(properties={
                "doc_id": row.doc_id,
                "title": row.title,
                "text": row.text,
                "url": row.url
                #"title_sparse": title_sparse_blobs[row.Index],
                #"title_colbert": title_colbert_blobs[row.Index],
            }, vector={  
                "title_dense": title_embeddings["dense_vecs"][row.Index],
                "text_dense": doc_embeddings["dense_vecs"][row.Index]
            }, uuid=row.doc_id)
            outer_progress.update(1)
        if b.number_errors != 0:
            print(f"Found Errors: {b.number_errors}")
        b.flush()
        

  0%|          | 0/1876367 [00:00<?, ?it/s]



Inference Embeddings:   0%|          | 0/40 [00:00<?, ?it/s]

Inference Embeddings:   2%|▎         | 1/40 [00:01<01:16,  1.97s/it]

Inference Embeddings:   5%|▌         | 2/40 [00:02<00:52,  1.38s/it]

Inference Embeddings:   8%|▊         | 3/40 [00:03<00:43,  1.17s/it]

Inference Embeddings:  10%|█         | 4/40 [00:04<00:37,  1.05s/it]

Inference Embeddings:  12%|█▎        | 5/40 [00:05<00:34,  1.02it/s]

Inference Embeddings:  15%|█▌        | 6/40 [00:06<00:31,  1.07it/s]

Inference Embeddings:  18%|█▊        | 7/40 [00:07<00:29,  1.13it/s]

Inference Embeddings:  20%|██        | 8/40 [00:08<00:27,  1.15it/s]

Inference Embeddings:  22%|██▎       | 9/40 [00:08<00:26,  1.19it/s]

Inference Embeddings:  25%|██▌       | 10/40 [00:09<00:24,  1.21it/s]

Inference Embeddings:  28%|██▊       | 11/40 [00:10<00:23,  1.24it/s]

Inference Embeddings:  30%|███       | 12/40 [00:11<00:21,  1.28it/s]

Inference Embeddings:  32%|███▎      | 13/40 [00:11<00:20,  1.31it/s]

Inference Embeddings: 

KeyboardInterrupt: 

In [81]:

zh = client.collections.get("neuclir_1_mutli_bge_m3")
aggregation = zh.aggregate.over_all(total_count=True)
print(aggregation.total_count)

487620


In [80]:
client.cluster.nodes(
    collection="neuclir_1_mutli_bge_m3",
    output="verbose"
)

[Node(git_hash='cfdbdd0', name='node1', shards=[Shard(collection='Neuclir_1_mutli_bge_m3', name='xhHerVp3Eq14', node='node1', object_count=487300, vector_indexing_status='READY', vector_queue_length=0, compressed=False, loaded=True)], stats=Stats(object_count=487300, shard_count=1), status='HEALTHY', version='1.27.5')]

In [4]:
from weaviate.collections.classes.grpc import MetadataQuery

query_embeddings = model.encode(["What political impact do news have on teenagers?"], return_dense=True, return_sparse=False, return_colbert_vecs=False)
response = zh.query.near_vector(near_vector=query_embeddings["dense_vecs"][0], target_vector="title_dense", limit=50, return_metadata=MetadataQuery(distance=True))

for o in response.objects:
    print(o.metadata.distance)

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


0.3497281074523926
0.3542219400405884
0.36670786142349243
0.3710690140724182
0.3710908889770508
0.379378080368042
0.38135790824890137
0.3862995505332947
0.38806724548339844
0.39258015155792236
0.394734263420105
0.39501863718032837
0.39628756046295166
0.3965839147567749
0.3984275460243225
0.39956390857696533
0.399910032749176
0.4027395248413086
0.40353405475616455
0.40462398529052734
0.406577467918396
0.40823471546173096
0.41130876541137695
0.4132359027862549
0.41399049758911133
0.41496437788009644
0.41694867610931396
0.4179171919822693
0.4182232618331909
0.4188825488090515
0.419994592666626
0.42144525051116943
0.4215751886367798
0.42173147201538086
0.4217910170555115
0.4219786524772644
0.4228867292404175
0.42291176319122314
0.42298340797424316
0.42474257946014404
0.425207257270813
0.42567533254623413
0.4257087707519531
0.42625755071640015
0.426602840423584
0.42671459913253784
0.4272133708000183
0.42755210399627686
0.4275628328323364
0.4281851649284363
